In [1]:
from transformers import ViTForImageClassification, ViTFeatureExtractor, TrainingArguments, Trainer
from datasets import load_dataset, Dataset
import torch
from torchvision import datasets, transforms
from pathlib import Path
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt

c:\Users\gabri\anaconda3\envs\Torch_NLP\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [41]:
train_dataset_path = "C:\MSAAI\AAI-521\Final Project Data\Transformer Data\Training Data"
train_metadata_csv = "C:\MSAAI\AAI-521\Final Project Data\Transformer Data\metadata_train.csv"

val_dataset_path = "C:\MSAAI\AAI-521\Final Project Data\Transformer Data\Validation Data"
val_metadata_csv = "C:\MSAAI\AAI-521\Final Project Data\Transformer Data\metadata_val.csv"

In [42]:
train_metadata = pd.read_csv("C:\MSAAI\AAI-521\Final Project Data\Transformer Data\metadata_train.csv")
val_metadata = pd.read_csv("C:\MSAAI\AAI-521\Final Project Data\Transformer Data\metadata_val.csv")

In [43]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5]),  # Normalize between -1 and 1
])

In [44]:
train_dataset = datasets.ImageFolder(root=train_dataset_path, transform=transform)
val_dataset = datasets.ImageFolder(root=val_dataset_path, transform=transform)

In [ ]:
model = ViTForImageClassification.from_pretrained(
    "google/vit-base-patch16-224",
    num_labels=len(train_dataset.classes),
    ignore_mismatched_sizes=True
)

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([7]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([7, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [46]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [47]:
model.to(device)

ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTSdpaAttention(
            (attention): ViTSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_fe

In [48]:
from PIL import Image

# Load the feature extractor
feature_extractor = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224")

# Function to preprocess datasets
def preprocess_dataset(dataset):
    image_paths = [sample[0] for sample in dataset.samples]
    labels = [sample[1] for sample in dataset.samples]

    def preprocess(images, labels):
        # Load images and preprocess
        processed_images = [Image.open(image_path).convert("RGB") for image_path in images]
        pixel_values = feature_extractor(processed_images, return_tensors="pt")["pixel_values"]
        return {"pixel_values": pixel_values, "labels": torch.tensor(labels)}

    dataset = Dataset.from_dict({"image_path": image_paths, "label": labels})
    return dataset.map(lambda x: preprocess(x["image_path"], x["label"]), batched=True)

# Preprocess training and validation datasets
train_hf_dataset = preprocess_dataset(train_dataset)
val_hf_dataset = preprocess_dataset(val_dataset)

c:\Users\gabri\anaconda3\envs\Torch_NLP\lib\site-packages\transformers\models\vit\feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(












 18%|█▊        | 500/2770 [29:44<2:15:00,  3.57s/it]


















Map: 100%|██████████| 1094/1094 [00:03<00:00, 284.34 examples/s]


In [ ]:
from transformers import TrainingArguments, Trainer

# Define training arguments
training_args = TrainingArguments(
    output_dir="./vit-finetune-results",
    evaluation_strategy="steps",
    eval_steps=500,
    save_steps=500,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    learning_rate=5e-5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=100,
    load_best_model_at_end=True,
    save_total_limit=2,
    fp16=True,  
    report_to="none",  
)

# Function to compute metrics, including per-class accuracy
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    preds = np.argmax(predictions, axis=1)
    report = classification_report(labels, preds, target_names=train_dataset.classes, output_dict=True)
    
    # Print per-class accuracy
    print("\nPer-Class Accuracy:")
    for class_name, metrics in report.items():
        if isinstance(metrics, dict):
            print(f"{class_name}: {metrics['precision'] * 100:.2f}%")

    # Return overall metrics for Trainer
    return {
        "accuracy": report["accuracy"],
        "precision": np.mean([metrics["precision"] for metrics in report.values() if isinstance(metrics, dict)]),
        "recall": np.mean([metrics["recall"] for metrics in report.values() if isinstance(metrics, dict)]),
        "f1": np.mean([metrics["f1-score"] for metrics in report.values() if isinstance(metrics, dict)]),
    }

# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_hf_dataset,
    eval_dataset=val_hf_dataset,
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
)

c:\Users\gabri\anaconda3\envs\Torch_NLP\lib\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
c:\Users\gabri\anaconda3\envs\Torch_NLP\lib\site-packages\accelerate\accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [50]:
# Fine-tune the model
trainer.train()

  4%|▎         | 100/2770 [01:19<34:01,  1.31it/s]

                                            


                                                  


  4%|▎         | 100/2770 [01:19<34:01,  1.31it/s]

{'loss': 0.8743, 'grad_norm': 8.041891098022461, 'learning_rate': 4.82129963898917e-05, 'epoch': 0.18}


  7%|▋         | 200/2770 [02:40<34:06,  1.26it/s]

                                            


                                                  


  7%|▋         | 200/2770 [02:40<34:06,  1.26it/s]

{'loss': 0.6424, 'grad_norm': 8.900994300842285, 'learning_rate': 4.640794223826715e-05, 'epoch': 0.36}


 11%|█         | 300/2770 [03:59<32:29,  1.27it/s]

                                            


                                                  


 11%|█         | 300/2770 [03:59<32:29,  1.27it/s]

...

 97%|█████████▋| 2700/2770 [40:46<00:58,  1.20it/s]

                                            


                                                   


 97%|█████████▋| 2700/2770 [40:46<00:58,  1.20it/s]

{'loss': 0.0007, 'grad_norm': 0.06750218570232391, 'learning_rate': 1.2815884476534297e-06, 'epoch': 4.87}


100%|██████████| 2770/2770 [41:42<00:00,  1.67it/s]

                                            


                                                   


100%|██████████| 2770/2770 [41:43<00:00,  1.11it/s]

{'train_runtime': 2503.285, 'train_samples_per_second': 17.705, 'train_steps_per_second': 1.107, 'train_loss': 0.19202751522944292, 'epoch': 5.0}


TrainOutput(global_step=2770, training_loss=0.19202751522944292, metrics={'train_runtime': 2503.285, 'train_samples_per_second': 17.705, 'train_steps_per_second': 1.107, 'total_flos': 3.4345988889388646e+18, 'train_loss': 0.19202751522944292, 'epoch': 5.0})

In [51]:
# Evaluate the model on validation dataset
metrics = trainer.evaluate()
print("Validation metrics:", metrics)

100%|██████████| 69/69 [00:54<00:00,  1.27it/s]


Per-Class Accuracy:
akiec: 73.91%
bcc: 76.47%
bkl: 90.16%
df: 83.33%
mel: 56.52%
nv: 95.38%
vasc: 85.71%
macro avg: 80.21%
weighted avg: 91.94%
Validation metrics: {'eval_loss': 0.2495461404323578, 'eval_accuracy': 0.9223034734917733, 'eval_precision': 0.8151769138234054, 'eval_recall': 0.7441569684876175, 'eval_f1': 0.7730059705785233, 'eval_runtime': 55.8367, 'eval_samples_per_second': 19.593, 'eval_steps_per_second': 1.236, 'epoch': 5.0}


In [52]:
# Print accuracy
print(f"Validation Accuracy: {metrics['eval_accuracy'] * 100:.2f}%")

Validation Accuracy: 92.23%


In [2]:
# Load the log history from the trainer
log_history = trainer.state.log_history

# Convert to a pandas DataFrame for easier manipulation
df = pd.DataFrame(log_history)

# Filter for training and validation losses
train_loss = df[df["loss"].notnull()]
eval_loss = df[df["eval_loss"].notnull()]

# Plot training and validation losses
plt.figure(figsize=(10, 6))
plt.plot(train_loss["step"], train_loss["loss"], label="Training Loss", marker="o")
plt.plot(eval_loss["step"], eval_loss["eval_loss"], label="Validation Loss", marker="o")

# Add labels, legend, and grid
plt.xlabel("Steps")
plt.ylabel("Loss")
plt.title("Learning Curve")
plt.legend()
plt.grid(True)
plt.show()

NameError: name 'trainer' is not defined

In [53]:
# Save the final model explicitly
trainer.save_model("./vit-final-model")